## Notebook: SW-General_Database_Check

#### Abstract
The purpose of this notebook is to run a generic database health check. This does not look for specific archetecture pre-requisites per se. The notebook does try to gather data for general healh, best practices, and key performance indicators. Various SQL was taken from other Jupyter Notebooks written by Ember Crooks or other performance and tuning articles, blogs, and resources. Various links to these methods are at the end of the notebook in the "Credit and References" section.

#### To Use This Notebook
- If the libraries have not been installed, you will need to manually excute the *Install Notebook Prerequisites* section and restart the kernel.
- At a minimum you will need to manually *Import the Notebook Prerequisites* to load the libraries needed (this needs to be done only once after the notebook is opened that specific time). 
- A variable file with connectivity information is expected to be held in the parent directory.
- Manually run the *Establish Database Connection*, type the password, and hit enter. 
- Once done you can select the next section and choose "Cells" from the menu bar and then "Run all Below".
- Note: The SQL/Code cells are hidden to make it easier to read the generated output. You can turn that off and on in the next section. I reccomend exposing the SQL untill the whole notebook runs and then I would toggle off the SQL.



---

## Code Toggle

In [ ]:
## Hide code cells
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<i><b>The raw code for this IPython notebook is by default hidden for easier reading.</b><br><br>
To see view the queries used in this notebook, you will need to toggle on/off the raw code, by clicking <a href="javascript:code_toggle()">here</a>.</i>''')

---

## Install Notebook Prerequisites

In [ ]:
# If this notebook is running on a Windows based machine with a Db2 client, you will need to run
# the following to allow you to run Db2 commands (i.e. db2exfmt) not basic SQL

import sys,os,os.path
os.environ['IBM_DB_HOME']='/Applications/dsdriver'

In [ ]:
# If you have not installed the following libraries in Python already (command line or via a notebook),
# you should run the following statements.

# Load importlib to give yourself the ability to import details from Python to see if certian libraries exist.
# !pip3 install importlib     #Import library
import importlib

# The following will check for ibm_db_sa as a litmus test. If it exists, we assume the other libraries are installed.
# If it does not exist, install the libraries.

spec = importlib.util.find_spec("ibm_db_sa")
if spec is None:
    print("Installing prerequisites.")
    # Dependancy: ipython-sql -> sqlalchemy -> ibm_db_sa -> ibm_db
    !pip3 install ipython-sql #SQL Magic (%sql) using SQLAlchemy connect strings
    !pip3 install ibm_db      #Db2 API for Python
    !pip3 install ibm_db_sa   #Db2 adaptor for SQLAlchemy
    !pip3 install matplotlib  #Plotting library for visualizations
    !pip3 install pandas      #Data analysis and manipulation
    
    # Unnofficial community extentions, creates a new tab in Jupyter Notebook
    # Uncheck "disable configuration for nbextensions without explicit compatibility" within new tab
    !pip3 install https://github.com/ipython-contrib/jupyter_contrib_nbextensions/tarball/master
    !jupyter contrib nbextension install --user
    print()
else:
    print("Prerequisites already met.")



## Import Notebook Perequisites

In [ ]:
# Import necessary modules and load SQL Magic
import ibm_db
import ibm_db_sa
import sqlalchemy
%load_ext sql
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates
from datetime import datetime
import pandas as pd
from IPython.display import display, HTML, Markdown
#import nbextensions
%matplotlib inline
import getpass

# Configure SQL Magic in a few nice ways
%config SqlMagic.style = 'MSWORD_FRIENDLY'
pd.set_option('display.max_rows', 4096)
pd.set_option('display.max_columns', 4096)

---

## Establish Target Database Connection

In [ ]:
# Define filename for passwords
filename = 'ember_variables.py'
# source the file
%run ../$filename

In [ ]:
# Connect to target Db2 database

# Connection Details
inst=OMS10_inst['PROD']
user=OMS10_user['PROD']
host=OMS10_host['PROD']
schema='OMSUSR'  # Not necessary for connection, but used later to focus target of SQL
db=OMS10_db['PROD']
port=OMS10_port['PROD']

# Prompt for Password
password = getpass.getpass('Enter password for '+user)
import urllib
password = urllib.parse.quote(password) #Use to handle special characters

# Connection String
%sql db2+ibm_db://$user:$password@$host:$port/$db


---

## Server Level Details

In [ ]:
%%sql db_server << 
SELECT 
    HOST_NAME,
    OS_NAME AS OS,
    OS_FULL_VERSION AS OS_VERSION,
    OS_ARCH_TYPE,
    CPU_ONLINE AS CPU,
    CPU_CORES_PER_SOCKET AS CORES_PER_SOCKET,
    MEMORY_TOTAL AS MEMORY,
    MEMORY_SWAP_TOTAL AS SWAP,
    MEMORY_SWAP_FREE AS SWAP_FREE
FROM TABLE(SYSPROC.ENV_GET_SYSTEM_RESOURCES()) AS T
WITH UR

In [ ]:
display(Markdown("#### Server Details"))
display(Markdown("Tip: Is swap space 1-2x the size of dedicated memory?"))
db_server_df=db_server.DataFrame()
db_server_df.columns=db_server_df.columns.str.upper()
db_server_df

---

## Db2 Version and Licensing

In [ ]:
%%sql db_ver << 
SELECT 
    INST_NAME, 
    SERVICE_LEVEL, 
    BLD_LEVEL, 
    FIXPACK_NUM 
FROM SYSIBMADM.ENV_INST_INFO 
WITH UR

In [ ]:
display(Markdown("#### Db2 Database Version"))
db_ver_df=db_ver.DataFrame()
db_ver_df.columns=db_ver_df.columns.str.upper()
db_ver_df

In [ ]:
%%sql db_license << 
SELECT 
    INSTALLED_PROD, 
    INSTALLED_PROD_FULLNAME, 
    LICENSE_INSTALLED ,
    PROD_RELEASE ,
    LICENSE_TYPE 
FROM SYSIBMADM.ENV_PROD_INFO 
WITH UR

In [ ]:
display(Markdown("#### Db2 Licensing"))
db_license_df=db_license.DataFrame()
db_license_df.columns=db_license_df.columns.str.upper()
db_license_df

---

## Metrics by Instance

In [ ]:
%%sql instance_uptime <<
SELECT 
    VARCHAR_FORMAT(DB2START_TIME, 'YYYY-MM-DD HH24:MM:SS') AS INSTANCE_ACTIVATION
    FROM TABLE(MON_GET_INSTANCE(-2))
WITH UR

In [ ]:
display(Markdown("#### Db2 Instance Start time"))
instance_uptime_df=instance_uptime.DataFrame()
instance_uptime_df.columns=instance_uptime_df.columns.str.upper()
instance_uptime_df

In [ ]:
%%sql db_env_settings <<
SELECT 
  REG_VAR_NAME AS DB2SET_PARM, 
  REG_VAR_VALUE AS DB2SET_VALUE
FROM TABLE(ENV_GET_REG_VARIABLES(-1, 1)) 
WHERE REG_VAR_NAME IN ('DB2COMM','DB2AUTH','DB2_PARALLEL_IO','DB2_RESTORE_GRANT_ADMIN_AUTHORITIES','AUTOSTART')
WITH UR

In [ ]:
display(Markdown("#### Common Db2 Environment Settings To Review For Best Practice"))
db_env_settings_df=db_env_settings.DataFrame()
db_env_settings_df.columns=db_env_settings_df.columns.str.upper()
db_env_settings_df

In [ ]:
%%sql mon_settings <<
SELECT 
  NAME AS MONITOR_VALUE,
  VALUE AS IS_ON
FROM SYSIBMADM.DBMCFG
WHERE NAME IN ('dft_mon_bufpool','dft_mon_lock','dft_mon_sort','dft_mon_stmt','dft_mon_table','dft_mon_timestamp','dft_mon_uow')

In [ ]:
display(Markdown("#### Are Database Metrics Being Monitored (DBM CFG)?"))
mon_settings_df=mon_settings.DataFrame()
mon_settings_df.columns=mon_settings_df.columns.str.upper()
mon_settings_df

In [ ]:
%%sql key_dbm_config <<
SELECT 
  NAME AS DBM_VALUE,
  VALUE AS CURRENT_SETTING
FROM SYSIBMADM.DBMCFG
WHERE NAME IN ('svcename','dftdbpath','diagpath','instance_memory','intra_parallel','keystore_type','keystore_location')

In [ ]:
display(Markdown("#### Key Database Manager Settings (DBM CFG)"))
key_dbm_config_df=key_dbm_config.DataFrame()
key_dbm_config_df.columns=key_dbm_config_df.columns.str.upper()
key_dbm_config_df

---

## Metrics by Database

In [ ]:
%%sql database_uptime <<
SELECT
VARCHAR_FORMAT(DB_CONN_TIME, 'YYYY-MM-DD HH24:MM:SS') AS DATABASE_ACTIVATION
    FROM TABLE(MON_GET_DATABASE(-2))
WITH UR

In [ ]:
display(Markdown("#### Database Start time"))
display(Markdown("Note: Many notebook calculations use cumulative database metrics collected from database start time. You could be looking at 2 days or 2 years of information. Metrics will change on database restart, which will reset the cumulative data."))
database_uptime_df=database_uptime.DataFrame()
database_uptime_df.columns=database_uptime_df.columns.str.upper()
database_uptime_df

In [ ]:
%%sql db_last_bkup <<
SELECT VARCHAR_FORMAT(LAST_BACKUP, 'YYYY-MM-DD HH24:MM:SS') AS LAST_DATABASE_BACKUP
    FROM TABLE(MON_GET_DATABASE(-2))
WITH UR;

In [ ]:
display(Markdown("#### Last Database Backup"))
db_last_bkup_df=db_last_bkup.DataFrame()
db_last_bkup_df.columns=db_last_bkup_df.columns.str.upper()
db_last_bkup_df

In [ ]:
%%sql key_db_cfg << 
SELECT 
  NAME AS DB_VALUE,
  VALUE AS CURRENT_SETTING
FROM SYSIBMADM.DBCFG 
WHERE NAME IN ('database_memory','locktimeout','logarchmeth1','logfilsiz','logprimary','logsecond','numlogspan','logarchcompr1','self_tuning_mem')

In [ ]:
display(Markdown("#### Key Database Settings (DB CFG)"))
key_db_cfg_df=key_db_cfg.DataFrame()
key_db_cfg_df.columns=key_db_cfg_df.columns.str.upper()
key_db_cfg_df

In [ ]:
%%sql db_arss <<
SELECT 
	ROWS_RETURNED,
	SELECT_SQL_STMTS,
	CASE 
		WHEN SELECT_SQL_STMTS > 0 
			THEN DECIMAL(FLOAT(rows_returned)/FLOAT(SELECT_SQL_STMTS),10,2)
			ELSE 'No Div. By zero'
	END AS AVG_RESULT_SET_SIZE,
	CASE 
		WHEN DECIMAL(FLOAT(rows_returned)/FLOAT(SELECT_SQL_STMTS),10,2) <= 10 
			THEN 'Transaction Processing Like Workload' 
			ELSE 'Warehouse Like Workload'
	END AS WORKLOAD_TYPE
FROM TABLE(MON_GET_DATABASE(-2))
WITH UR

In [ ]:
display(Markdown("#### Database Average Result Set Size (ARSS)"))
display(Markdown("Tip: DB ARSS for OLTP < 10 | DB ARSS for DW > 10"))
db_arss_df=db_arss.DataFrame()
db_arss_df.columns=db_arss_df.columns.str.upper()
db_arss_df

In [ ]:
%%sql db_ixref <<
SELECT ROWS_READ, 
 ROWS_RETURNED,
 ROWS_READ/ROWS_RETURNED AS IXREF 
FROM TABLE(MON_GET_WORKLOAD('',-2)) AS T 
WHERE WORKLOAD_NAME='SYSDEFAULTUSERWORKLOAD' 
WITH UR

In [ ]:
display(Markdown("#### Database Index Read Efficiency (IXREF)"))
display(Markdown("Tip: (OLTP)  <10 Ideal | 10-100 Potentially Acceptiable  | 100-1000 Poor | >1000 Bad"))
db_ixref_df=db_ixref.DataFrame()
db_ixref_df.columns=db_ixref_df.columns.str.upper()
db_ixref_df

In [ ]:
%%sql db_rollbackpercent <<
SELECT TOTAL_APP_COMMITS AS APP_COMMITS, 
	   TOTAL_APP_ROLLBACKS AS APP_ROLLBACKS,
	   TOTAL_APP_COMMITS + TOTAL_APP_ROLLBACKS AS TOTAL,
	   CASE
            WHEN TOTAL_APP_COMMITS + TOTAL_APP_ROLLBACKS > 0 THEN
                DECIMAL(FLOAT(TOTAL_APP_ROLLBACKS)/FLOAT(TOTAL_APP_COMMITS + TOTAL_APP_ROLLBACKS),10,2)*100
            ELSE 0
        END AS PCNT_ROLLBACK
FROM TABLE(MON_GET_DATABASE(-2))
WITH UR

In [ ]:
display(Markdown("#### Database Transaction Rollback Percentage"))
display(Markdown("Tip: Percentage of your hardware is wasting time on transactions that never complete. Anything over 1% would be uncommon. Can lead to locking and contention in database."))
db_rollbackpercent_df=db_rollbackpercent.DataFrame()
db_rollbackpercent_df.columns=db_rollbackpercent_df.columns.str.upper()
db_rollbackpercent_df

In [ ]:
db_rollback= %sql SELECT TOTAL_APP_COMMITS AS APP_COMMITS, \
	TOTAL_APP_ROLLBACKS AS APP_ROLLBACKS \
FROM TABLE(MON_GET_DATABASE(-2)) \
WITH UR

db_rollback_df=db_rollback.DataFrame()
db_rollback_df.sum().plot(kind='pie', autopct='%1.0f%%', ylabel=" ", title='Percent of Database Rollbacks')


---

## Metrics by Table

In [ ]:
%%sql table_byreadoverflow << 
WITH t AS (
 SELECT
      TABSCHEMA AS SCHEMA,
      TABNAME AS TABLE,
      ROWS_READ,
      OVERFLOW_ACCESSES,
      CASE WHEN ROWS_READ > 0 
        THEN DECIMAL(FLOAT(OVERFLOW_ACCESSES)/FLOAT(ROWS_READ),10,2)*100 
        ELSE 0 
      END AS R_OVRFLW_PCT
 FROM TABLE(MON_GET_TABLE('','',-2)) AS t WHERE TABSCHEMA =:schema AND OVERFLOW_ACCESSES > 0)
SELECT * FROM t WHERE R_OVRFLW_PCT >= 3 ORDER BY R_OVRFLW_PCT DESC
WITH UR

In [ ]:
display(Markdown("#### Tables > 3% Read Overflows"))
display(Markdown("Tip: Read overflow over 3% is a signal to run a REORG. Db2 is looking for data where it expects to be and finds a pointer to a new location causing double logical read."))
table_byreadoverflow_df=table_byreadoverflow.DataFrame()
table_byreadoverflow_df.columns=table_byreadoverflow_df.columns.str.upper()
table_byreadoverflow_df

In [ ]:
%%sql table_bywriteoverflow << 
WITH t AS (
 SELECT
      TABSCHEMA AS SCHEMA,
      TABNAME AS TABLE,
      ROWS_UPDATED,
      OVERFLOW_CREATES,
      CASE WHEN ROWS_UPDATED > 0 
        THEN DECIMAL(FLOAT(OVERFLOW_CREATES)/FLOAT(ROWS_UPDATED),10,2)*100 
        ELSE 0 
      END AS W_OVRFLW_PCT
 FROM TABLE(MON_GET_TABLE('','',-2)) AS t WHERE TABSCHEMA =:schema AND OVERFLOW_CREATES > 0)
SELECT * FROM t WHERE W_OVRFLW_PCT >= 3 ORDER BY W_OVRFLW_PCT DESC
WITH UR

In [ ]:
display(Markdown("#### Tables > 3% Write Overflows"))
display(Markdown("Tip: Write overflow over 3% is a signal to run a REORG. An update operation to VARCHAR causes the field to be longer and not fit on original data page. A pointer is left and the data is written on another page."))
table_bywriteoverflow_df=table_bywriteoverflow.DataFrame()
table_bywriteoverflow_df.columns=table_bywriteoverflow_df.columns.str.upper()
table_bywriteoverflow_df

In [ ]:
%%sql table_top10_byinsert << 
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_INSERTED) DESC) AS RANK, 
       TRIM(TABSCHEMA) as SCHEMA, 
       TRIM(TABNAME) as TABLE, 
       SUM(ROWS_INSERTED) as ROWS_INSERTED
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Inserted"))
table_top10_byinsert_df=table_top10_byinsert.DataFrame()
table_top10_byinsert_df.columns=table_top10_byinsert_df.columns.str.upper()
table_top10_byinsert_df

In [ ]:
%%sql table_top10_byread << 
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_READ) DESC) AS RANK, 
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS TABLE, 
       SUM(ROWS_READ) AS ROWS_READ
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB 
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Read"))
table_top10_byread_df=table_top10_byread.DataFrame()
table_top10_byread_df.columns=table_top10_byread_df.columns.str.upper()
table_top10_byread_df

In [ ]:
%%sql table_top10_byupdate <<
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_UPDATED) DESC) AS RANK, 
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS TABNAME, 
       SUM(ROWS_UPDATED) AS ROWS_UPDATED
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB 
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Updated"))
table_top10_byupdate_df=table_top10_byupdate.DataFrame()
table_top10_byupdate_df.columns=table_top10_byupdate_df.columns.str.upper()
table_top10_byupdate_df

In [ ]:
%%sql table_top10_bydelete <<
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(ROWS_DELETED) DESC) AS RANK,
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS NAME, 
       SUM(ROWS_DELETED) AS ROWS_DELETED
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB
      WHERE TABSCHEMA=:schema
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Rows Deleted"))
table_top10_bydelete_df=table_top10_bydelete.DataFrame()
table_top10_bydelete_df.columns=table_top10_bydelete_df.columns.str.upper()
table_top10_bydelete_df

In [ ]:
%%sql table_top10_byscan <<
SELECT * 
FROM (SELECT ROW_NUMBER() OVER(ORDER BY SUM(TABLE_SCANS) DESC) AS RANK, 
       TRIM(TABSCHEMA) AS SCHEMA, 
       TRIM(TABNAME) AS TABLE, 
       SUM(TABLE_SCANS) AS NUM_SCANS
      FROM TABLE(MON_GET_TABLE('','',-2)) AS TB 
      GROUP BY TABSCHEMA, TABNAME) 
WHERE RANK<=10
WITH UR

In [ ]:
display(Markdown("#### Top 10 Tables by Number of Scans"))
table_top10_byscan_df=table_top10_byscan.DataFrame()
table_top10_byscan_df.columns=table_top10_byscan_df.columns.str.upper()
table_top10_byscan_df

In [ ]:
%%sql top_lowcard_indexes <<

-- Find top 25 busiest tables on 100 rows or more
-- Calculate index cardinality percentage compared to total table cardinality
-- Show index cardinality percentage less than to equal to 3%
-- Do not take (U)nique indexes into account because they are used to enforce uniqueness

SELECT A.TABSCHEMA AS SCHEMA,
       A.TABNAME AS TABLE,
       A.INDNAME AS INDEX_NAME,
       B.CARD AS TB_CARD,
       A.FULLKEYCARD AS IX_FULLKEYCARD,
       INT((FLOAT(A.FULLKEYCARD)/FLOAT(B.CARD)) * 100) AS PCNT_TBCARD
FROM SYSCAT.INDEXES A INNER JOIN SYSCAT.TABLES B
       ON A.TABSCHEMA = B.TABSCHEMA
       AND A.TABNAME = B.TABNAME
WHERE A.FULLKEYCARD > 0
     AND A.TABSCHEMA <> 'SYSIBM'
 	 AND B.CARD > 100
 	 AND A.UNIQUERULE <> 'U'
 	 AND INT((FLOAT(A.FULLKEYCARD)/FLOAT(B.CARD)) * 100) <= 3
 	 AND A.TABSCHEMA=:schema
 	 AND A.TABNAME IN 
	   (SELECT TABNAME
       FROM TABLE(MON_GET_TABLE('','',-2)) 
       WHERE TABSCHEMA=:schema
       ORDER BY ROWS_INSERTED + ROWS_UPDATED + ROWS_DELETED DESC
       FETCH FIRST 25 ROWS ONLY)
ORDER BY INT((FLOAT(A.FULLKEYCARD)/FLOAT(B.CARD)) * 100) ASC, B.CARD DESC
WITH UR

In [ ]:
display(Markdown("#### Low Cardinality Indexes (of Top 25 Busiest Tables)"))
display(Markdown("Tip: The higher the FULLKEYCARD is (in relation to table cardinality) the more unique the value is. This leads to an efficient index. The lower FULLKEYCARD value means less unique values (1 being all values the same) which causes an inefficient index."))
top_lowcard_indexes_df=top_lowcard_indexes.DataFrame()
top_lowcard_indexes_df.columns=top_lowcard_indexes_df.columns.str.upper()
top_lowcard_indexes_df

In [ ]:
%%sql top_tables_unused_indexes <<
SELECT
	TABSCHEMA,
	TABNAME,
	COUNT(INDNAME) AS IDX_NOT_USED_CNT
FROM SYSCAT.INDEXES
WHERE LASTUSED='0001-01-01' AND tabschema=:schema
AND TABNAME IN
 	   (SELECT TABNAME
       FROM TABLE(MON_GET_TABLE('','',-2))
       WHERE TABSCHEMA=:schema
       ORDER BY ROWS_INSERTED + ROWS_UPDATED + ROWS_DELETED DESC
       FETCH FIRST 25 ROWS ONLY)
GROUP BY TABSCHEMA,TABNAME
ORDER BY IDX_NOT_USED_CNT DESC
WITH UR

In [ ]:
display(Markdown("#### Number of Indexes That Have Never Been Used (of Top 25 Busiest Tables)"))
display(Markdown("Tip: The more indexes a table has, the more of a performance impact on INSERT operations. These tables have high write activity but have indexes that are never used and could be slowing down INSERT performance."))
top_tables_unused_indexes_df=top_tables_unused_indexes.DataFrame()
top_tables_unused_indexes_df.columns=top_tables_unused_indexes_df.columns.str.upper()
top_tables_unused_indexes_df

---

## Metrics by Statement

In [ ]:
%%sql stmt_top10_byixref <<
WITH SUM_TAB (SUM_RR) AS (
        SELECT FLOAT(SUM(ROWS_READ))
        FROM TABLE(MON_GET_PKG_CACHE_STMT ( 'D', NULL, NULL, -2)) AS T)
SELECT
        INSERT_TIMESTAMP AS FIRST_INSRT_INTO_CACHE,
        SUBSTR(STMT_TEXT,1,50) AS STATEMENT,
        ROWS_READ,
        DECIMAL(100*(FLOAT(ROWS_READ)/SUM_TAB.SUM_RR),5,2) AS PCT_TOT_RR,
        ROWS_RETURNED,
        CASE
            WHEN ROWS_RETURNED > 0 THEN
                DECIMAL(FLOAT(ROWS_READ)/FLOAT(ROWS_RETURNED),10,2)
            ELSE -1
        END AS IXREF,
        NUM_EXECUTIONS
    FROM TABLE(MON_GET_PKG_CACHE_STMT ( 'D', NULL, NULL, -2)) AS T, SUM_TAB
    ORDER BY ROWS_READ DESC FETCH FIRST 10 ROWS ONLY 
    WITH UR

In [ ]:
display(Markdown("#### Top 10 Inefficient Statements (IXREF)"))
display(Markdown("Tip: Index Read Efficiency shows how many rows are scanned to fetch the row DB2 needs. Ideally in a OLTP environment this should be <10. High IXREF SQL that executes frequently could be a performance problem. Keep in mind, the timestamp of first insert into package cache can change on rerun as things are flushed out, etc. The data you are seeing are details since that first insert into the cache."))
stmt_top10_byixref_df=stmt_top10_byixref.DataFrame()
stmt_top10_byixref_df.columns=stmt_top10_byixref_df.columns.str.upper()
stmt_top10_byixref_df

---

## Database Wait Time

In [ ]:
%%sql db_timespent << select * 
    from table(mon_get_database(-2));

In [ ]:
df=db_timespent.DataFrame()
df.columns= df.columns.str.upper()
df.columns

In [ ]:
# Input: 'df' is a DataFrame that has time series data from MON_GET_DATABASE.
# Because the Wait Times are cumulative, I really only care about the most 
# recent entry in 'df'.
#
# Get the index of the last row, and build a dictionary with with all of the 
# wait time elements:
i = 0
ix = {'AGENT_WAIT_TIME': df['AGENT_WAIT_TIME'][i],
      'WLM_QUEUE_TIME_TOTAL': df['WLM_QUEUE_TIME_TOTAL'][i],
      'LOCK_WAIT_TIME':  df['LOCK_WAIT_TIME'][i], 
      'LOG_BUFFER_WAIT_TIME': df['LOG_BUFFER_WAIT_TIME'][i],
      'LOG_DISK_WAIT_TIME': df['LOG_DISK_WAIT_TIME'][i],
      'TCPIP_RECV_WAIT_TIME': df['TCPIP_RECV_WAIT_TIME'][i],
      'TCPIP_SEND_WAIT_TIME': df['TCPIP_SEND_WAIT_TIME'][i],
      'IPC_RECV_WAIT_TIME': df['IPC_RECV_WAIT_TIME'][i],
      'IPC_SEND_WAIT_TIME': df['IPC_SEND_WAIT_TIME'][i],
      'FCM_RECV_WAIT_TIME': df['FCM_RECV_WAIT_TIME'][i],
      'FCM_SEND_WAIT_TIME': df['FCM_SEND_WAIT_TIME'][i],
      'AUDIT_SUBSYSTEM_WAIT_TIME': df['AUDIT_SUBSYSTEM_WAIT_TIME'][i],
      'AUDIT_FILE_WRITE_WAIT_TIME': df['AUDIT_FILE_WRITE_WAIT_TIME'][i],
      'DIAGLOG_WRITE_WAIT_TIME': df['DIAGLOG_WRITE_WAIT_TIME'][i],
      'POOL_READ_TIME': df['POOL_READ_TIME'][i],
      'POOL_WRITE_TIME': df['POOL_WRITE_TIME'][i],
      'DIRECT_READ_TIME': df['DIRECT_READ_TIME'][i],
      'DIRECT_WRITE_TIME': df['DIRECT_WRITE_TIME'][i],
      'EVMON_WAIT_TIME': df['EVMON_WAIT_TIME'][i],
      'TOTAL_EXTENDED_LATCH_WAIT_TIME': df['TOTAL_EXTENDED_LATCH_WAIT_TIME'][i],
      'PREFETCH_WAIT_TIME': df['PREFETCH_WAIT_TIME'][i],
      'COMM_EXIT_WAIT_TIME': df['COMM_EXIT_WAIT_TIME'][i],
      'IDA_SEND_WAIT_TIME': df['IDA_SEND_WAIT_TIME'][i],
      'IDA_RECV_WAIT_TIME': df['IDA_RECV_WAIT_TIME'][i],
      'CF_WAIT_TIME': df['CF_WAIT_TIME'][i],
      'RECLAIM_WAIT_TIME': df['RECLAIM_WAIT_TIME'][i],
      'SPACEMAPPAGE_RECLAIM_WAIT_TIME': df['SPACEMAPPAGE_RECLAIM_WAIT_TIME'][i]}

# Convert the dictionary into a new DataFrame.
myx=pd.DataFrame.from_dict(ix, orient='index')

# Calcuate the percentage of overall wait for each value.
total=myx[0].sum()
myx['PCT'] = myx.apply(lambda x: x[0]/total, axis=1)

# Filter out wait elements that have <= 0.5% wait time), to avoid having 
# 27 slices of pie, most of which are tiny slivers
myx = myx[myx['PCT'] >= 0.005]

# Set up the graph
display(Markdown("#### Database Wait Time"))
xlabel=myx.index.tolist()
yname=myx.columns.values[0]
plt = myx.plot(kind='pie',y=yname,labels=xlabel,figsize=[5,5],autopct='%0.1f%%')
plt.get_legend().remove()
plt.axes.get_yaxis().set_visible(False)

---

## Locking Analysis (Past 24 Hours)

In [ ]:
%%sql lock_event_24 <<
SELECT
  SUBSTR(EVENT_TYPE,1,18) AS EVENT_TYPE,
  COUNT(*)/2 AS COUNT
FROM DBAMON.LOCK_EVENT
WHERE EVENT_TIMESTAMP >= CURRENT TIMESTAMP - 24 hours
GROUP BY EVENT_TYPE 
WITH UR

In [ ]:
display(Markdown("####  Locking Events - Past 24 hours"))
lock_event_24_df=lock_event_24.DataFrame()
lock_event_24_df.columns=lock_event_24_df.columns.str.upper()
lock_event_24_df

In [ ]:
%%sql lock_event_bytable_24 <<
SELECT
  SUBSTR(LP.TABLE_SCHEMA,1,18) AS TABLE_SCHEMA,
  SUBSTR(LP.TABLE_NAME,1,30) AS TABLE_NAME,
  SUBSTR(LE.EVENT_TYPE,1,18) AS LOCK_EVENT,
  COUNT(*)/2 AS COUNT
FROM DBAMON.LOCK_PARTICIPANTS LP, DBAMON.LOCK_EVENT LE
WHERE LP.XMLID=LE.XMLID 
	AND EVENT_TIMESTAMP >= CURRENT TIMESTAMP - 24 HOURS
	AND NOT (LP.TABLE_SCHEMA IS NULL OR LP.TABLE_SCHEMA IS NULL)
GROUP BY LP.TABLE_SCHEMA, LP.TABLE_NAME, LE.EVENT_TYPE
ORDER BY LP.TABLE_SCHEMA, LP.TABLE_NAME, LE.EVENT_TYPE 
WITH UR

In [ ]:
display(Markdown("####  Locking Events By Table - Past 24 hours"))
lock_event_bytable_24_df=lock_event_bytable_24.DataFrame()
lock_event_bytable_24_df.columns=lock_event_bytable_24_df.columns.str.upper()
lock_event_bytable_24_df

## Security and Access 

In [ ]:
%%sql public_auth <<
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'DATABASE CONNECT AUTHORITY DETECTED' AS ACCESS_LEVEL 
   FROM SYSCAT.DBAUTH 
   WHERE GRANTEE='PUBLIC' AND CONNECTAUTH='Y'
 UNION
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'TABLE LEVEL AUTHORITY DETECTED' AS ACCESS_LEVEL 
   FROM SYSCAT.TABAUTH WHERE GRANTEE='PUBLIC'
   AND TABSCHEMA NOT LIKE 'SYS%'
   AND (CONTROLAUTH='Y' OR ALTERAUTH='Y' OR DELETEAUTH='Y' 
   OR DELETEAUTH='Y' OR INDEXAUTH='Y' OR INSERTAUTH='Y' 
   OR REFAUTH='Y' OR SELECTAUTH='Y' OR UPDATEAUTH='Y')
 UNION
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'PACKAGE LEVEL AUTHORITY DETECTED' AS ACCESS_LEVEL FROM SYSCAT.PACKAGEAUTH WHERE GRANTEE='PUBLIC' AND CONTROLAUTH='Y'
 UNION
SELECT DISTINCT GRANTEE, GRANTEETYPE, 'SCHEMA LEVEL AUTHORITY DETECTED' AS ACCESS_LEVEL FROM SYSCAT.SCHEMAAUTH WHERE GRANTEE='PUBLIC'
   AND SCHEMANAME NOT LIKE 'SYS%'
   AND (ALTERINAUTH='Y' OR CREATEINAUTH='Y' OR DROPINAUTH='Y' 
   OR SELECTINAUTH='Y' OR INSERTINAUTH='Y' OR UPDATEINAUTH='Y' 
   OR DELETEINAUTH='Y' OR EXECUTEINAUTH='Y' OR SCHEMAADMAUTH='Y' 
   OR ACCESSCTRLAUTH='Y' OR DATAACCESSAUTH='Y' OR LOADAUTH='Y')
ORDER BY GRANTEE, GRANTEETYPE, ACCESS_LEVEL
WITH UR

In [ ]:
display(Markdown("####  PUBLIC Access & Authorities"))
display(Markdown("Tip: Sometimes PUBLIC access is found and is OK (i.e. ability to bind). What we are looking for is the ability for PUBLIC to connect and read or update sensitive information."))
public_auth_df=public_auth.DataFrame()
public_auth_df.columns=public_auth_df.columns.str.upper()
public_auth_df

---

## Error Logs

In [ ]:
%%sql error_72hours <<
SELECT 
 TIMESTAMP, 
 DBNAME,
 APPL_ID, 
 LEVEL, 
 IMPACT, 
 SUBSTR(MSG,1, 70) AS MSG 
FROM TABLE (PD_GET_DIAG_HIST( 'MAIN', 'D', '', NULL, NULL) ) AS T 
WHERE LEVEL IN ('C','S','E') 
      AND MSG IS NOT NULL 
      AND TIMESTAMP >= current timestamp - 72 HOURS
ORDER BY TIMESTAMP DESC
WITH UR

In [ ]:
display(Markdown("#### Error Log - Past 72 hours"))
display(Markdown("Data below is not exhaustive. This is showing CRITICAL, SEVERE, or ERROR message in the db2diag.log. "))
error_72hours_df=error_72hours.DataFrame()
error_72hours_df.columns=error_72hours_df.columns.str.upper()
error_72hours_df

---

## Credit & References

- Ragu on Tech: [Db2 Finding Top 10 Most Active Tables](https://www.raghu-on-tech.com/2020/02/29/db2-finding-top-10-most-active-tables/)
- Xtivia: [Index Read Efficiency a KPI for Db2](https://www.virtual-dba.com/blog/index-read-efficiency-db2/)
- Datageek: [Db2 Table Scans](https://datageek.blog/en/2012/11/28/db2-table-scans/)
- DBI Software Blog: [Table Read I/O and Overflows](https://www.dbisoftware.com/blog/db2_performance.php?id=116)
- Xtivia: [Index Read Efficiency - A KPI for Db2](https://www.virtual-dba.com/blog/index-read-efficiency-db2/)
- Datageek: [Boost Your Performance – (IREF) Index Read Efficiency](https://datageek.blog/en/2014/03/17/boost-your-performance-iref-index-read-efficiency/)
- Use the Index Luke: [More Indexes, Slower Insert](https://use-the-index-luke.com/sql/dml/insert)
- Statsology: [How to Create a Pie Chart from Pandas Dataframe](https://www.statology.org/pandas-pie-chart/)